In [1]:
! pip install -U spacy -q

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 29.1/29.1 MB 58.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.7/3.7 MB 79.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.2/9.2 MB 91.1 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
en-core-web-sm 3.7.1 requires spacy<3.8.0,>=3.7.2, but you have spacy 3.8.3 which is incompatible.


In [2]:
!python -m spacy info

/usr/local/lib/python3.10/dist-packages/spacy/util.py:910: UserWarning: [W095] Model 'en_core_web_sm' (3.7.1) was trained with spaCy v3.7.2 and may not be 100% compatible with the current version (3.8.3). If you see errors or degraded performance, download a newer compatible model or retrain your custom model with the current spaCy version. For more details and available updates, run: python -m spacy validate
  warnings.warn(warn_msg)

============================== Info about spaCy ==============================

spaCy version    3.8.3                         
Location         /usr/local/lib/python3.10/dist-packages/spacy
Platform         Linux-6.1.85+-x86_64-with-glibc2.35
Python version   3.10.12                       
Pipelines        en_core_web_sm (3.7.1)        



In [3]:
import spacy
from spacy.tokens import DocBin
from tqdm import tqdm

nlp = spacy.blank("en") # load a new spacy model
db = DocBin() # create a DocBin object

In [8]:
import json
f = open('annotations _final.json')
TRAIN_DATA = json.load(f)

In [30]:
TRAIN_DATA

[['Rajesh Patel, a 42-year-old Financial Advisor, was the central figure in a high-profile financial crime case.\r',
  {'entities': [[0, 12, 'PERSON'], [16, 27, 'AGE'], [28, 45, 'PROFESSION']]}],
 ['Rajesh Patel had been employed at IndoGlobal Insurance Ltd. for over a decade.\r',
  {'entities': [[0, 12, 'PERSON'], [34, 59, 'ORG'], [71, 77, 'TIME']]}],
 ['Rajesh Patel and hansal patel were known for his expertise in investment strategies and financial management.\r',
  {'entities': [[0, 12, 'PERSON'], [17, 29, 'PERSON']]}],
 ['The accused Rajesh is alleged to have used his position to siphon funds through a series of forged documents and unauthorized transactions between 2021 and 2022.\r',
  {'entities': [[12, 18, 'PERSON'], [147, 151, 'DATE'], [156, 160, 'DATE']]}],
 ['The alleged crimes were uncovered when Meera Smith, an Internal Auditor at National Bank of Commerce, noticed discrepancies during a routine financial review in January 2023.\r',
  {'entities': [[39, 50, 'PERSON'],
    

In [35]:
for text, annot in tqdm(TRAIN_DATA):
    doc = nlp.make_doc(text)
    ents = []
    for start, end, label in annot['entities']:
        span = doc.char_span(start, end, label=label, alignment_mode="contract")
        if span is None:
            print("Skipping entity")
        else:
            ents.append(span)
    doc.ents = ents
    db.add(doc)

db.to_disk("./training_data.spacy")

100%|██████████| 100/100 [00:00<00:00, 3377.95it/s]

Skipping entity
Skipping entity
Skipping entity


In [14]:
! python -m spacy init config config.cfg --lang en --pipeline ner --optimize efficiency

⚠ To generate a more effective transformer-based config (GPU-only),
install the spacy-transformers package and re-run this command. The config
generated now does not use transformers.
ℹ Generated config template specific for your use case
- Language: en
- Pipeline: ner
- Optimize for: efficiency
- Hardware: CPU
- Transformer: None
✔ Auto-filled config with all values
✔ Saved config
config.cfg
You can now add your data and train your pipeline:
python -m spacy train config.cfg --paths.train ./train.spacy --paths.dev ./dev.spacy


In [19]:
! python -m spacy train config.cfg --output ./ --paths.train ./training_data.spacy --paths.dev ./training_data.spacy

ℹ Saving to output directory: .
ℹ Using CPU

=========================== Initializing pipeline ===========================
✔ Initialized pipeline

============================= Training pipeline =============================
ℹ Pipeline: ['tok2vec', 'ner']
ℹ Initial learn rate: 0.001
E    #       LOSS TOK2VEC  LOSS NER  ENTS_F  ENTS_P  ENTS_R  SCORE 
---  ------  ------------  --------  ------  ------  ------  ------
  0       0          0.00     41.32    0.00    0.00    0.00    0.00
  8     200        858.36   4226.97   76.24   76.38   76.10    0.76
 18     400        210.59    643.52  100.00  100.00  100.00    1.00
 30     600         31.82     32.93  100.00  100.00  100.00    1.00
 46     800          5.43      4.65  100.00  100.00  100.00    1.00
 65    1000         21.35     12.28  100.00  100.00  100.00    1.00
 89    1200          8.27      7.32  100.00  100.00  100.00    1.00
117    1400         79.84     17.92  100.00  100.00  100.00    1.00
152    1600        316.74    116.71 

In [20]:
nlp_ner = spacy.load("/content/model-best")

In [33]:
doc = nlp_ner('''Dr. Jane Smith, a renowned cardiologist at Mayo Clinic, presented a new study on heart disease treatment during the World Health Summit in Geneva on October 5, 2024. The research, funded by the American Heart Association, highlighted significant improvements in treatment outcomes using a new drug. The findings are expected to revolutionize the approach to cardiovascular health and reduce heart disease mortality rates globally.''')

In [34]:
spacy.displacy.render(doc, style="ent", jupyter=True) # display in Jupyter